In [ ]:
import tensorflow as tf
from tensorflow.keras.models import load_model
import cv2
import matplotlib.pyplot as plt
from tqdm import tqdm
import pandas as pd
import numpy as np
import glob
import os
import sys
sys.path.append('segmentation-cnn')
sys.path.append('classification-cnn')
from dataset import tf_dataset
from model import jacard_coef,jacard_coef_loss,DiceLoss
from segment_imageV2 import segmented_image,plate_characters,plate_characters_only_classification,four_point_transform,segmented_image_warped_skimageV2
import imutils

In [ ]:
"""
Notebook auxiliar empleado para guardar los resutlados de X modelos de la red clasificación diferentes para comprar las diferencias entre cada uno de ellos
"""

In [ ]:
# Load Model Segmentation List
model_seg_list = glob.glob('../data/segmentation-cnn/models/*')
model_seg_list.sort()
# Load Model Classification List
model_clas_list = glob.glob('../data/classification-cnn/models/*')
model_clas_list.sort()

In [ ]:
model_clas_list

In [ ]:

model1 = load_model(model_seg_list[9],custom_objects={'jacard_coef_loss' : jacard_coef_loss,'jacard_coef' : jacard_coef, 'DiceLoss' : DiceLoss})
model2 = load_model(model_seg_list[20],custom_objects={'jacard_coef_loss' : jacard_coef_loss,'jacard_coef' : jacard_coef, 'DiceLoss' : DiceLoss})
model3 = load_model(model_seg_list[14],custom_objects={'jacard_coef_loss' : jacard_coef_loss,'jacard_coef' : jacard_coef, 'DiceLoss' : DiceLoss})
model4 = load_model(model_seg_list[13],custom_objects={'jacard_coef_loss' : jacard_coef_loss,'jacard_coef' : jacard_coef, 'DiceLoss' : DiceLoss})
model5 = load_model(model_seg_list[11],custom_objects={'jacard_coef_loss' : jacard_coef_loss,'jacard_coef' : jacard_coef, 'DiceLoss' : DiceLoss})
# model_classification = load_model(model_clas_list[0])

In [ ]:
model1 = load_model(model_clas_list[3])
model2 = load_model(model_clas_list[4])
model3 = load_model(model_clas_list[5])
model4 = load_model(model_clas_list[10])

In [ ]:
def read_image(image_path,mode): 
    org = cv2.imread(image_path, mode)
    x = org / 255.0
    x = x.astype(np.float32)
    return x, org

def bounding_box(img):
    bbox = []
    img_split = img.split('-')[3].split('_')
    for coor in img_split:
        bbox.append((int(coor.split('&')[0]),int(coor.split('&')[1])))
        # bbox = bbox[::-1]
    return np.float32([bbox[2],bbox[3],bbox[1],bbox[0]])

col_names = ['img','mask']
df = pd.read_csv ('segmentation-cnn/original csv/test_dataset.csv',sep=',',header=None,names=col_names)
images = df['img'].tolist()
masks = df['mask'].tolist()


test_dataset = tf_dataset(images, masks, batch_size=16, buffer_size=1000)

In [ ]:
perspective, org = read_image('/home/mihai/git/TFG/code/output pred/24/6. perspective.jpg',cv2.IMREAD_COLOR)
resultSeg,divs,bw,cleared = segmented_image_warped_skimageV2(perspective)

In [ ]:
plt.imshow(bw)
cv2.imwrite('bw.jpg',bw)

In [ ]:

index = 1

path = '/home/mihai/git/TFG/code/predsModels/classification/'

for index,imgPath in enumerate(images[0:15]):
    dr = path + str(index+1)
    os.mkdir(dr)
    dr = dr + '/'
    
    image = cv2.imread(images[index])
    # image = cv2.cvtColor(image,cv2.COLOR_BGR2RGB)
    box = bounding_box(images[index].split('/')[-1])

    perspective, _ = four_point_transform(image.copy(), box)

    he, wi,dm = perspective.shape
    div = int(wi/7)

    divs = [i * div + 5 for i in range(1,7)]
    divs[0] = divs[0] - 5
    divs[1] = divs[1] - 5
    divs[-2] = divs[-2] - 5
    divs[-1] = divs[-1] - 5

    resultSeg = perspective.copy()
    for i in divs:
        # divImgList.append(resultSeg)
        resultSeg[:,i-1:i+1] = 0

    divs.insert(0,0)
    segImg = []

    for ind in range(len(divs)):
        if(ind == len(divs)-1):
            segImg.append(resultSeg[:,divs[ind]:,:])
        else:
            segImg.append(resultSeg[:,divs[ind]:divs[ind+1],:])
    cv2.imwrite(dr + 'perspective.png',resultSeg)
    for index2,item in enumerate(segImg):
        cv2.imwrite(dr + str(index2+1) + '.jpg',item)

# plt.imshow(perspective)
# plt.imshow(segImg[-1])

# GUARDAR LAS IMAGENES Y LAS SEG. SIRVEN PARA LAS DOS PARTES. GOLD EN ESTE CASO 
# EN CASO DE PARTE SEGMENTACION COMENTAR SKIIMAGE, FINDCOUNTURS,ETC....


In [ ]:
path = '/home/mihai/git/TFG/code/predsModels/classification/'

lst = glob.glob(path + '*')
lst.sort()
lr = glob.glob(lst[0] + '/*.jpg')
lr.sort()
lst

In [ ]:
classes = ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9','A', 'B', 'C', 'D', 'E', 'F', 'G', 
               'H', 'J', 'K', 'L', 'M', 'N', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X','Y', 'Z']
    
path = '/home/mihai/git/TFG/code/predsModels/classification/'

lst = glob.glob(path + '*')
lst.sort()

modelList=[model1,model2,model3,model4]

for item in lst: # Para todas las carpetas
    subList = glob.glob(item + '/*.jpg')
    subList.sort()
    matricula = []
    arrayRes = []
    for letterPath in subList[1:]:
        letter = cv2.imread(letterPath)
        letter = cv2.resize(letter,(32,32),cv2.INTER_CUBIC)
        letter = cv2.cvtColor(letter,cv2.COLOR_BGR2RGB)
        letter = np.expand_dims(letter,axis=0)
        pred = model4.predict(letter)
        # x = [str(i) for i in pred[0]]
        # arrayRes.append(",".join(x))
        matricula.append(classes[np.argmax(pred)])
    dst = item + '/model4_' + "".join(matricula) 
    # np.savetxt(dst,arrayRes)
    np.savetxt(dst,[])